# Process data

In [1]:
import os

In [2]:
data_dir = os.path.join('..','resources','data')

In [3]:
os.listdir(data_dir)

['05-2019.csv',
 '06-2019.csv',
 '07-2019.csv',
 '08-2019.csv',
 '09-2019.csv',
 '10-2019.csv',
 '11-2019.csv',
 '12-2019.csv',
 'airports.csv']

In [4]:
import pandas as pd

In [5]:
primary_df = pd.concat([
    pd.read_csv(os.path.join(data_dir,filename))
    for filename in os.listdir(data_dir)[:-1]
])

primary_df_rows, primary_df_cols = primary_df.shape

print(f"{primary_df_rows:,} rows × {primary_df_cols:,} columns")

5,512,903 rows × 35 columns


In [6]:
primary_df.head()

,carrier_code,flight_number,origin_airport,destination_airport,date,scheduled_elapsed_time,tail_number,departure_delay,arrival_delay,delay_carrier,...,HourlyPrecipitation_x,HourlyStationPressure_x,HourlyVisibility_x,HourlyWindSpeed_x,STATION_y,HourlyDryBulbTemperature_y,HourlyPrecipitation_y,HourlyStationPressure_y,HourlyVisibility_y,HourlyWindSpeed_y
0,AS,121,SEA,ANC,2019-05-01,215,N615AS,-8,-16,0,...,0.0,29.59,10.0,8.0,7.027253e+10,42.0,0.0,30.16,10.0,3.0
1,F9,402,LAX,DEN,2019-05-01,147,N701FR,17,-4,0,...,0.0,29.65,10.0,3.0,7.256500e+10,34.0,0.0,24.43,4.0,0.0
2,F9,662,SFO,DEN,2019-05-01,158,N346FR,44,27,0,...,0.0,29.98,10.0,6.0,7.256500e+10,34.0,0.0,24.43,4.0,0.0
3,F9,790,PDX,DEN,2019-05-01,156,N332FR,24,10,0,...,0.0,29.98,10.0,0.0,7.256500e+10,34.0,0.0,24.43,4.0,0.0
4,AS,108,ANC,SEA,2019-05-01,210,N548AS,-9,-31,0,...,0.0,30.18,10.0,5.0,7.279302e+10,44.0,0.0,29.58,10.0,7.0


In [44]:
# Get data types and number of null values for each column
pd.concat(
    [
        primary_df.dtypes,
        primary_df.isna().sum()
    ],
    axis=1,
    keys=['dtypes','null_count']
)

,dtypes,null_count
carrier_code,object,0
flight_number,int64,0
origin_airport,object,0
destination_airport,object,0
date,object,0
scheduled_elapsed_time,int64,0
tail_number,object,6973
departure_delay,int64,0
arrival_delay,int64,0
delay_carrier,int64,0


**Note:** Eventually, `cancelled_code` will be our target column for a machine-learning algorithm. Because the column in the source data is `cancelled_code` and not `canceled_code`, the double-l spelling will be used in this work.

In [8]:
# Combine `year`, `month`, and `day` into a single `string` in the same format as the `date` column
# and check for equality against the actual date column.

# Check only a few rows
(
    primary_df[['year','month','day']][:3]
    .apply(lambda row: '-'.join([val.zfill(2) for val in row.values.astype(str)]), axis=1)
    .equals(
        primary_df.date[:3]
    )
)

# Check all the rows
# (
#     primary_df[['year','month','day']]
#     .apply(lambda row: '-'.join([val.zfill(2) for val in row.values.astype(str)]), axis=1)
#     .equals(
#         primary_df.date
#     )
# )

True

**Note:** The full check, above, is commented out because it takes a long time, but if run, it does show equality between the entire `date` series and the combined `year`-`month`-`day` series.

Because `year`, `month`, and `day` were originally stored as `int64` values, this also tells us that all the values in `date` are properly formatted (no leading or trailing spaces, *etc*.).

In [9]:
primary_df.cancelled_code.value_counts()

N    5426150
B      41919
A      23451
C      21370
D         13
Name: cancelled_code, dtype: int64

### What do the codes mean?

According to the United States Department of Transportation Bureau of Transportation Statistics Airlines and Airports data, [Number 14 - On-Time Reporting](https://www.bts.gov/topics/airlines-and-airports/number-14-time-reporting):

**CANCELLATION CODES**
- `A`-Carrier Caused
- `B`-Weather
- `C`-National Aviation System
- `D`-Security

\[`N` is not on the list and represents "None" or "Not cancelled".\]

We are only interested in flights that were cancelled due to weather, so we will keep only rows with `cancelled_code` `B` or `N`.

In [11]:
primary_df = primary_df.loc[primary_df.cancelled_code.isin(['B','N'])]

primary_df_rows, primary_df_cols = primary_df.shape

print(f"{primary_df_rows:,} rows × {primary_df_cols:,} columns")

5,468,069 rows × 35 columns


In [12]:
primary_df.cancelled_code.value_counts()

N    5426150
B      41919
Name: cancelled_code, dtype: int64

In [36]:
primary_df.rename(columns={'cancelled_code':'cancelled'},inplace=True)

In [43]:
# Convert `cancelled_code` column into boolean `cancelled` column, where
# `B` = True (*was* cancelled) and `N` = False (*was not* cancelled)

try:
    print("Converting cancelled_code column to boolean… ", end="")
    primary_df.cancelled_code = (primary_df.cancelled_code == 'B')
    print()
except AttributeError:
    print("Column has already been processed.")

primary_df.cancelled.value_counts()

Converting cancelled_code column to boolean… Column has already been processed.


False    5426150
True       41919
Name: cancelled, dtype: int64

In [69]:
# How many flights were cancelled|not cancelled vs. how many departed|arrived

departed = ~primary_df.actual_departure_dt.isna()
arrived = ~primary_df.actual_arrival_dt.isna()

mult_ix = pd.MultiIndex.from_tuples([
    ('departed',True),
    ('departed',False),
    ('arrived',True),
    ('arrived',False),
])


pd.DataFrame(
    data=[
        [
            primary_df.loc[(~primary_df.cancelled) & (departed)].shape[0],
            primary_df.loc[(primary_df.cancelled) & (departed)].shape[0]
        ],
        [
            primary_df.loc[(~primary_df.cancelled) & (~departed)].shape[0],
            primary_df.loc[(primary_df.cancelled) & (~departed)].shape[0]
        ],
        [
            primary_df.loc[(~primary_df.cancelled) & (arrived)].shape[0],
            primary_df.loc[(primary_df.cancelled) & (arrived)].shape[0]
        ],
        [
            primary_df.loc[(~primary_df.cancelled) & (~arrived)].shape[0],
            primary_df.loc[(primary_df.cancelled) & (~arrived)].shape[0]
        ]
    ],
    index=mult_ix,
    columns=['not_cancelled','cancelled']
)

not_cancelled  cancelled
departed True         5426150       1854
         False              0      40065
arrived  True         5424261          0
         False           1889      41919

In [102]:
# Does anything stand out for cancelled flights that still departed?
primary_df.loc[primary_df.cancelled & departed].head().transpose()

,16715,17002,17815,18640,18750
carrier_code,AA,AA,AA,AA,AA
flight_number,1393,346,2761,1271,5821
origin_airport,OKC,DFW,DFW,IAH,DFW
destination_airport,DFW,MSY,STL,DFW,ELP
date,2019-05-01,2019-05-01,2019-05-01,2019-05-01,2019-05-01
scheduled_elapsed_time,69,85,105,75,104
tail_number,N751UW,N357PV,N971TW,N898NN,N243LR
departure_delay,176,83,111,113,28
arrival_delay,0,0,0,0,0
delay_carrier,0,0,0,0,0


In [103]:
# What about non-cancelled flights that didn't arrive?
primary_df.loc[~primary_df.cancelled & ~arrived].head().transpose()

,5154,12535,13657,16277,17368
carrier_code,AS,AA,WN,WN,UA
flight_number,55,2028,2272,2212,6296
origin_airport,SCC,MEM,PDX,ABQ,IAD
destination_airport,BRW,DFW,DAL,DAL,DFW
date,2019-05-01,2019-05-01,2019-05-01,2019-05-01,2019-05-01
scheduled_elapsed_time,45,99,230,105,209
tail_number,N609AS,N749US,N931WN,N788SA,N87353
departure_delay,29,398,-2,-5,212
arrival_delay,0,0,0,0,0
delay_carrier,0,0,0,0,0


In [104]:
primary_obj_cols = primary_df.select_dtypes('object').columns

primary_obj_cols.tolist()

['carrier_code',
 'origin_airport',
 'destination_airport',
 'date',
 'tail_number',
 'scheduled_departure_dt',
 'scheduled_arrival_dt',
 'actual_departure_dt',
 'actual_arrival_dt']

In [105]:
primary_df[[
    'date',
    'scheduled_departure_dt',
    'scheduled_arrival_dt',
    'actual_departure_dt',
    'actual_arrival_dt'
]].head()

,date,scheduled_departure_dt,scheduled_arrival_dt,actual_departure_dt,actual_arrival_dt
0,2019-05-01,2019-05-01 00:40:00,2019-05-01 03:15:00,2019-05-01 00:32:00,2019-05-01 02:59:00
1,2019-05-01,2019-05-01 00:59:00,2019-05-01 04:26:00,2019-05-01 01:16:00,2019-05-01 04:22:00
2,2019-05-01,2019-05-01 00:50:00,2019-05-01 04:28:00,2019-05-01 01:34:00,2019-05-01 04:55:00
3,2019-05-01,2019-05-01 00:55:00,2019-05-01 04:31:00,2019-05-01 01:19:00,2019-05-01 04:41:00
4,2019-05-01,2019-05-01 00:10:00,2019-05-01 04:40:00,2019-05-01 00:01:00,2019-05-01 04:09:00
